# Paraphrase

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/paraphrase](https://github.com/huseinzol05/Malaya/tree/master/example/paraphrase).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

In [1]:
%%time

import malaya
from pprint import pprint

CPU times: user 5.15 s, sys: 1.03 s, total: 6.17 s
Wall time: 7.43 s


In [2]:
malaya.paraphrase.available_transformer()

INFO:root:tested on 1k paraphrase texts.


,Size (MB),Quantized Size (MB),BLEU
t2t,832.0,279.0,0.59612
small-t2t,379.0,120.0,0.65849
t5,1250.0,481.0,0.86698
small-t5,355.6,195.0,0.81801


### Load T5 models

```python
def transformer(model: str = 't2t', quantized: bool = False, **kwargs):
    """
    Load Malaya transformer encoder-decoder model to generate a paraphrase given a string.

    Parameters
    ----------
    model : str, optional (default='t2t')
        Model architecture supported. Allowed values:

        * ``'t2t'`` - Malaya Transformer BASE parameters.
        * ``'small-t2t'`` - Malaya Transformer SMALL parameters.
        * ``'t5'`` - T5 BASE parameters.
        * ``'small-t5'`` - T5 SMALL parameters.

    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model. 
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result: malaya.model.tf.Paraphrase class
    """
```

In [3]:
t5 = malaya.paraphrase.transformer(model = 't5', quantized = True)

### Paraphrase simple string

We only provide `greedy_decoder` method for T5 models,

```python
@check_type
def greedy_decoder(self, string: str, split_fullstop: bool = True):
    """
    paraphrase a string. Decoder is greedy decoder with beam width size 1, alpha 0.5 .

    Parameters
    ----------
    string: str
    split_fullstop: bool, (default=True)
        if True, will generate paraphrase for each strings splitted by fullstop.

    Returns
    -------
    result: str
    """
```

In [4]:
string = "Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak daripada wujud isu digunakan terhadap Najib."
pprint(string)

('Beliau yang juga saksi pendakwaan kesembilan berkata, ia bagi mengelak '
 'daripada wujud isu digunakan terhadap Najib.')


In [5]:
pprint(t5.greedy_decoder([string]))

['Ini juga bagi mengelakkan kekeliruan yang biasa berlaku terhadap Najib.']


In [3]:
string = """
PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu.

Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.

Bekas Setiausaha Agung Bersatu Datuk Marzuki Yahya berkata, pada mesyuarat itu MPT sebulat suara menolak peletakan jawatan Dr Mahathir.

"Jadi ini agak berlawanan dengan keputusan yang kita sudah buat. Saya tak faham bagaimana Jabatan Pendaftar Pertubuhan Malaysia (JPPM) kata peletakan jawatan itu sah sedangkan kita sudah buat keputusan di dalam mesyuarat, bukan seorang dua yang buat keputusan.

"Semua keputusan mesti dibuat melalui parti. Walau apa juga perbincangan dibuat di luar daripada keputusan mesyuarat, ini bukan keputusan parti.

"Apa locus standy yang ada pada Setiausaha Kerja untuk membawa perkara ini kepada JPPM. Seharusnya ia dibawa kepada Setiausaha Agung sebagai pentadbir kepada parti," katanya kepada Harian Metro.

Beliau mengulas laporan media tempatan hari ini mengenai pengesahan JPPM bahawa Dr Mahathir tidak lagi menjadi Pengerusi Bersatu berikutan peletakan jawatannya di tengah-tengah pergolakan politik pada akhir Februari adalah sah.

Laporan itu juga menyatakan, kedudukan Muhyiddin Yassin memangku jawatan itu juga sah.

Menurutnya, memang betul Dr Mahathir menghantar surat peletakan jawatan, tetapi ditolak oleh MPT.

"Fasal yang disebut itu terpakai sekiranya berhenti atau diberhentikan, tetapi ini mesyuarat sudah menolak," katanya.

Marzuki turut mempersoal kenyataan media yang dibuat beberapa pimpinan parti itu hari ini yang menyatakan sokongan kepada Perikatan Nasional.

"Kenyataan media bukanlah keputusan rasmi. Walaupun kita buat 1,000 kenyataan sekali pun ia tetap tidak merubah keputusan yang sudah dibuat di dalam mesyuarat. Kita catat di dalam minit apa yang berlaku di dalam mesyuarat," katanya.
"""

In [4]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(string)
splitted = malaya.text.function.split_into_sentences(string)

In [5]:
' '.join(splitted[:2])

'PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu. Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.'

In [12]:
t5.greedy_decoder([' '.join(splitted[:2])])

['Peletakan jawatan Tun Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia ditolak di Mesyuarat Khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu. Tidak kira sama ada peletakan jawatan itu sah atau tidak, kerana sudah pun dinyatakan pada peringkat parti yang menyetujui semua, termasuk Presiden Tan Sri Muhyiddin Yassin.']

### Load Transformer

To load 8-bit quantized model, simply pass `quantized = True`, default is `False`.

We can expect slightly accuracy drop from quantized model, and not necessary faster than normal 32-bit float model, totally depends on machine.

In [7]:
model = malaya.paraphrase.transformer(model = 'small-t2t')
quantized_model = malaya.paraphrase.transformer(model = 'small-t2t', quantized = True)

#### Predict using greedy decoder

```python
def greedy_decoder(self, strings: List[str], **kwargs):
    """
    Paraphrase strings using greedy decoder.

    Parameters
    ----------
    strings: List[str]

    Returns
    -------
    result: List[str]
    """
```

In [13]:
' '.join(splitted[:2])

'PELETAKAN jawatan Tun Dr Mahathir Mohamad sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak di dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu. Justeru, tidak timbul soal peletakan jawatan itu sah atau tidak kerana ia sudah pun diputuskan pada peringkat parti yang dipersetujui semua termasuk Presiden, Tan Sri Muhyiddin Yassin.'

In [14]:
model.greedy_decoder([' '.join(splitted[:2])])

['Catatan mengenai peletakan jawatan Tun Dr. Mahathir sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) telah ditolak dalam mesyuarat khas MPT pada 24 Februari lalu, dan tidak akan ada persoalan bahawa ia sudah pun diputuskan pada peringkat parti yang menyetujui semua, termasuk Presiden Tan Sri Muhyiddin Yassin.']

In [15]:
quantized_model.greedy_decoder([' '.join(splitted[:2])])

['Tidak timbul persoalan bahawa peletakan jawatan Tun Dr. Mahathir sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) telah ditolak dalam mesyuarat khas MPT pada 24 Februari lalu, dan tidak ada persoalan bahawa ia sudah pun diputuskan pada peringkat parti yang menyetujui semua, termasuk Presiden Tan Sri Muhyiddin Yassin.']

#### Predict using beam decoder

```python
def beam_decoder(self, strings: List[str], **kwargs):
    """
    Paraphrase strings using beam decoder, beam width size 3, alpha 0.5 .

    Parameters
    ----------
    strings: List[str]

    Returns
    -------
    result: List[str]
    """
```

In [12]:
model.beam_decoder([' '.join(splitted[:2])])

['Catatan mengenai peletakan jawatan Tun Dr. Mahathir sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) telah ditolak pada 24 Februari lalu dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT).']

In [13]:
quantized_model.beam_decoder([' '.join(splitted[:2])])

['Catatan mengenai peletakan jawatan Tun Dr. Mahathir sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) telah ditolak pada 24 Februari lalu dalam mesyuarat khas Majlis Pimpinan Tertinggi (MPT).']

#### Predict using nucleus decoder

```python
def nucleus_decoder(self, strings: List[str], top_p: float = 0.7, **kwargs):
    """
    Paraphrase strings using nucleus sampling.

    Parameters
    ----------
    strings: List[str]
    top_p: float, (default=0.7)
        cumulative distribution and cut off as soon as the CDF exceeds `top_p`.

    Returns
    -------
    result: List[str]
    """
```

In [14]:
model.nucleus_decoder([' '.join(splitted[:2])])

['Pengaturan jawatan Tun Dr. Mahathir sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) ditolak dalam mesyuarat khas MPT pada 24 Februari lalu, dan tidak ada alasan untuk future pencalonan, kerana ia sudah pun diputuskan pada peringkat parti yang menyetujui semua, termasuk Presiden Tan Sri Muhyiddin Yassin.']

In [16]:
quantized_model.nucleus_decoder([' '.join(splitted[:2])], top_p = 0.5)

['Pemerintah dilantik sebagai Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu) oleh Majlis Pimpinan Tertinggi (MPT) pada 24 Februari lalu, dan tidak ada persoalan bahawa peletakan jawatan itu sah atau tidak, kerana sudah diputuskan pada peringkat parti yang menyetujui semua, termasuk Presiden, Tan Sri Muhyiddin Yassin. Kata Dr Mahathir Mohamad, "18." Parti itu telah menolak sebarang keputusan mengenai peletakan jawatan mereka. kerana semua itu tidak ada persoalan bahawa Muhyiddin Yassin. Parti Pribumi Bersatu Malaysia, yang menjauhkan diri dari semua haknya untuk jawatan mereka dari jawatannya. dan peletakan jawatan Tun Dr Mahathir Mohamad. parlimen, dalam kedudukan Dr Mahathir Mohamad, dalam peletakan jawatan Dr Mahathir Mohamad, menyuarakan pentadbiran, yang diterima oleh Majlis Pimpinan']